In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from collections import defaultdict
import random

from sklearn.feature_extraction.text import TfidfVectorizer
import urllib
import scipy.optimize
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /home/j1feng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
rent = pd.read_csv('clean.csv')
rent.review_summary = rent.review_summary.fillna('')
rent.review_text = rent.review_text.fillna('')

In [3]:
rent = rent.sample(frac=1).reset_index(drop=True)
train = rent[:180000]
valid = rent[180000:]

In [4]:
y_train = train.fit
y_valid = valid.fit

In [5]:
#np.unique(rent['rented for'])

In [6]:
#np.unique(rent['body type'])

In [7]:
#np.unique(rent['bust_char'])

In [8]:
#np.unique(rent['category'])

In [9]:
#rent[['age']].to_numpy()

In [10]:
# rented_for_one= pd.get_dummies(rent['rented for'], prefix='rented for')
# category_one = pd.get_dummies(rent['category'], prefix='category')
# bust_char_one = pd.get_dummies(rent['bust_char'], prefix='bust_char')
# body_type_one = pd.get_dummies(rent['body type'], prefix='body type')

In [18]:
def combine_feature(numpy_arrays):
    res = []
    for i in numpy_arrays:
        res += list(i.T)
    return np.array(res).T

In [11]:
#rented_for
# rented_for_train = rented_for_one[:180000].to_numpy()
# rented_for_valid = rented_for_one[180000:].to_numpy()

#category
# category_one_train = category_one[:180000].to_numpy()
# category_one_valid = category_one[180000:].to_numpy()

#bust_num
# bust_num_train = train[['bust_num']].to_numpy()
# bust_num_valid = valid[['bust_num']].to_numpy()

#bust_char
# bust_char_for_train = bust_char_one[:180000].to_numpy()
# bust_char_for_valid = bust_char_one[180000:].to_numpy()

#body_type
# body_type_train = body_type_one[:180000].to_numpy()
# body_type_valid = body_type_one[180000:].to_numpy()

#age
# age_train = train[['age']].to_numpy()
# age_valid = valid[['age']].to_numpy()

# size
# size_train = train[['size']].to_numpy()
# size_valid = valid[['size']].to_numpy()

# #weight
# weight_train = train[['weight']].to_numpy()
# weight_valid = valid[['weight']].to_numpy()

# #height
# height_train = train[['height']].to_numpy()
# height_valid = valid[['height']].to_numpy()

# #rating
rating_train = train[['rating']].to_numpy()
rating_valid = valid[['rating']].to_numpy()


# Train (tf-idf)

In [24]:
tfidf = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_summary = tfidf.fit_transform(train.review_summary).toarray()

In [25]:
tfidf2 = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_text = tfidf2.fit_transform(train.review_text).toarray()

In [26]:
combined_feat = combine_feature([Train_X_summary, Train_X_text, rating_train])

# Validation

In [27]:
Valid_X_summary = tfidf.transform(valid.review_summary).toarray()

In [28]:
Valid_X_text = tfidf2.transform(valid.review_text).toarray()

In [29]:
other_feature_valid = rating_valid

In [30]:
combined_valid = combine_feature([Valid_X_summary, Valid_X_text, other_feature_valid])

In [ ]:
clf = LogisticRegression(C=100)
clf.fit(combined_feat, y_train)

In [ ]:
clf.score(combined_valid, y_valid)

In [22]:
clf_fr = RandomForestClassifier()
clf_fr.fit(combined_feat, y_train)
clf_fr.score(combined_valid, y_valid)

0.8093112244897959

In [28]:
for d in [3, 5, 8, 10, 12]:
    clf_fr = RandomForestClassifier(max_depth = d)
    clf_fr.fit(combined_feat, y_train)
    print(d, clf_fr.score(combined_valid, y_valid))

3 0.7443399234693877
5 0.7443399234693877
8 0.7449776785714286
10 0.7488042091836735
12 0.7510363520408163


# Best scores

Ken's best scores; no paramter;
0.81728

In [29]:
#clf.score(combined_valid, y_valid)

0.8172831632653061

# Train (bag of words)

In [15]:
count = CountVectorizer(stop_words = 'english', max_features = 1000, tokenizer = word_tokenize) 
Train_X_summary = count.fit_transform(train.review_summary).toarray()

In [16]:
count2 = CountVectorizer(stop_words = 'english', max_features = 1000, tokenizer = word_tokenize) 
Train_X_text = count2.fit_transform(train.review_text).toarray()

In [19]:
combined_feat = combine_feature([Train_X_summary, Train_X_text, rating_train])

In [20]:
Valid_X_summary = count.transform(valid.review_summary).toarray()
Valid_X_text = count2.transform(valid.review_text).toarray()
other_feature_valid = rating_valid
combined_valid = combine_feature([Valid_X_summary, Valid_X_text, other_feature_valid])

In [21]:
clf = LogisticRegression(max_iter = 10000)
clf.fit(combined_feat, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
clf.score(combined_valid, y_valid)

0.8125

# More codes of ken being big brain

In [31]:
np.unique(rent['fit'])

array(['fit', 'large', 'small'], dtype=object)

In [32]:
rent_temp = rent

In [39]:
rent_temp['fit'][1]

'large'

In [41]:
for i in rent_temp.index:
    if rent_temp.loc[i,'fit'] == 'small':
        rent_temp.loc[i,'fit'] = 0
    if rent_temp.loc[i,'fit'] == 'fit':
        rent_temp.loc[i,'fit'] = 1
    if rent_temp.loc[i,'fit'] == 'large':
        rent_temp.loc[i,'fit'] = 2

In [47]:
rent_temp[['fit','rating']].corr()

,rating
rating,1.0


In [49]:
rent_temp["fit"] = rent_temp["fit"].astype(float)

In [50]:
column_1 = rent_temp["fit"]
column_2 = rent_temp["rating"]
correlation = column_1.corr(column_2)

In [51]:
correlation

0.028358140500538136

In [20]:
y_valid.value_counts()

fit      9235
small    1698
large    1611
Name: fit, dtype: int64

In [21]:
9235 / (len(y_valid))

0.7362085459183674

In [22]:
clf = LogisticRegression(max_iter = 10000, C=0.1)
clf.fit(combined_text, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
clf.score(combined_text_valid, y_valid)

0.8038903061224489